In [1]:
from functools import partial

from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import utils

from scripts import run
from utils import mapping
from utils import messages as ms

In [2]:
#use a class to define your input and output in order to have mutable variables
class gfc_process_IO:
    def __init__(self):
        #set up your inputs
        self.threshold = 30
        
        #set up output
        self.visualization = False
        self.clip_map = None
        self.csv_file = None
        
gfc_process_io = gfc_process_IO()

In [3]:
#create a map time
id_ = "gfc_process_widget"
title = "Map"

gfc_m = mapping.init_gfc_map()
gfc_map = wf.Tile(id_, title, inputs=[gfc_m])

In [4]:
import ipyvuetify as v

#create the output alert 
gfc_viz_output = wf.OutputWidget(ms.GFC_MESSAGE)

#create the button that start your process
gfc_viz_btn = wf.ProcessBtn('Update data')

#create the widgets following ipyvuetify requirements 
gfc_viz_slider = v.Slider(
    label= 'Threshold', 
    class_="mt-5", 
    thumb_label='always', 
    v_model=30,
)

gfc_viz_inputs = [gfc_viz_slider]

#bind the widget to the inputs
wb.bind(gfc_viz_slider, gfc_process_io, 'threshold', gfc_viz_output)

#create a process tile
id_ = "gfc_process_widget"
title = 'GFC visualization'

gfc_viz = wf.Tile(
    id_, 
    title, 
    btn=gfc_viz_btn, 
    inputs=gfc_viz_inputs, 
    output=gfc_viz_output
)

#bind the button to the process by writing a custom function
def process_start(widget, event, data, output, m):
    
    #debug
    asset = 'users/bornToBeAlive/aoi_PU'
    #threshold = 20
    #visualization = True
    
    #asset = gfc_aoi_IO.assetId
    threshold = gfc_process_io.threshold
    visualization = gfc_process_io.visualization
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #check inputs
    if not wb.checkInput(asset, output, ms.NO_AOI): return utils.toggleLoading(widget)
    if not wb.checkInput(threshold, output, ms.NO_THRESHOLD): return utils.toggleLoading(widget)
    
    #display the results 
    run.displayGfcMap(asset, threshold, m, visualization, output)
    
    #validate the visualization process
    setattr(gfc_process_io, 'visualization', True)
    
    #toggle the loading button
    return utils.toggleLoading(widget)

gfc_viz_btn.on_event('click', partial(
    process_start, 
    output=gfc_viz_output,
    m=gfc_m,
))

In [5]:
#wrap the tiles together in a layout row 
gfc_map.md6=True
gfc_viz.md6=True

gfc_viz_process = v.Layout(
    Row=True,
    xs12=True, 
    align_center=True,
    children = [gfc_viz,gfc_map]
)



In [6]:
#this tiles will only be displayed if you launch voila from this file 
gfc_viz_process

Layout(align_center=True, children=[Layout(align_center=True, children=[Card(children=[Html(children=['Map'], …